In [2]:
import os
import glob
from PIL import Image
import shutil
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.utils import image_dataset_from_directory
import cv2

In [3]:
# Load the dataset from the CSV file
file_path = '/kaggle/input/satellite-imagery/train_classes.csv'
df = pd.read_csv(file_path)

df.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [4]:
df['image_name'] = df['image_name'] + '.jpg'

df.head()

,image_name,tags
0,train_0.jpg,haze primary
1,train_1.jpg,agriculture clear primary water
2,train_2.jpg,clear primary
3,train_3.jpg,clear primary
4,train_4.jpg,agriculture clear habitation primary road


In [5]:
# Extract unique labels
all_labels = set(label for sublist in df['tags'].str.split() for label in sublist)

all_labels

{'agriculture',
 'artisinal_mine',
 'bare_ground',
 'blooming',
 'blow_down',
 'clear',
 'cloudy',
 'conventional_mine',
 'cultivation',
 'habitation',
 'haze',
 'partly_cloudy',
 'primary',
 'road',
 'selective_logging',
 'slash_burn',
 'water'}

In [6]:
new_df = pd.DataFrame(df['image_name'])

# Create columns for each unique label and initialize with zeros
for label in all_labels:
    new_df[label] = 0

# Perform one-hot encoding
for i, row in df.iterrows():
    tags = row['tags'].split()
    for tag in tags:
        new_df.at[i, tag] = 1

new_df = new_df[['image_name', 'clear', 'partly_cloudy', 'cloudy', 'haze', 'primary', 'water', 'habitation', 'agriculture', 'road', 'cultivation', 'bare_ground', 'slash_burn', 'selective_logging', 'blooming', 'blow_down', 'conventional_mine', 'artisinal_mine']]
new_df.head()

,image_name,clear,partly_cloudy,cloudy,haze,primary,water,habitation,agriculture,road,cultivation,bare_ground,slash_burn,selective_logging,blooming,blow_down,conventional_mine,artisinal_mine
0,train_0.jpg,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
1,train_1.jpg,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0
2,train_2.jpg,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,train_3.jpg,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,train_4.jpg,1,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0


In [7]:
atm_data = new_df[['image_name', 'clear', 'partly_cloudy', 'cloudy', 'haze']]

atm_data.head()

,image_name,clear,partly_cloudy,cloudy,haze
0,train_0.jpg,0,0,0,1
1,train_1.jpg,1,0,0,0
2,train_2.jpg,1,0,0,0
3,train_3.jpg,1,0,0,0
4,train_4.jpg,1,0,0,0


In [8]:
land_data = new_df[['image_name', 'primary', 'water', 'habitation', 'agriculture', 'road', 'cultivation', 'bare_ground', 'slash_burn', 'selective_logging', 'blooming', 'blow_down', 'conventional_mine', 'artisinal_mine']]

# Rename 'primary' to 'primary_rainforest' in land_data
land_data = land_data.rename(columns={'primary': 'primary_rainforest'})
land_data.head()

,image_name,primary_rainforest,water,habitation,agriculture,road,cultivation,bare_ground,slash_burn,selective_logging,blooming,blow_down,conventional_mine,artisinal_mine
0,train_0.jpg,1,0,0,0,0,0,0,0,0,0,0,0,0
1,train_1.jpg,1,1,0,1,0,0,0,0,0,0,0,0,0
2,train_2.jpg,1,0,0,0,0,0,0,0,0,0,0,0,0
3,train_3.jpg,1,0,0,0,0,0,0,0,0,0,0,0,0
4,train_4.jpg,1,0,1,1,1,0,0,0,0,0,0,0,0


In [9]:
# Split the DataFrame into train and test sets
atm_train_data, atm_test_data = train_test_split(atm_data, test_size=0.05, random_state=42)
land_train_data, land_test_data = train_test_split(land_data, test_size=0.05, random_state=42)

In [12]:
# Path to the folder containing the images
image_folder = '/kaggle/input/satellite-imagery/train-jpg'

# Create folders for train and test sets if they don't exist
train_folder = 'train'
test_folder = 'test'

os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Move images for the train set
for index, row in atm_train_data.iterrows():
    filename = row['image_name']
    src_path = os.path.join(image_folder, filename)
    dst_path = os.path.join(train_folder, filename)
    shutil.copy(src_path, dst_path)

# Move images for the test set
for index, row in atm_test_data.iterrows():
    filename = row['image_name']
    src_path = os.path.join(image_folder, filename)
    dst_path = os.path.join(test_folder, filename)
    shutil.copy(src_path, dst_path)

In [13]:
num_classes = len(atm_train_data.columns) - 1  # Excluding the 'image_name' column

In [14]:
def fbeta(y_true, y_pred, beta = 2, epsilon = 1e-4):

    beta_squared = beta**2
    
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(tf.greater(tf.cast(y_pred, tf.float32), tf.constant(0.5)), tf.float32)
    
    tp = tf.reduce_sum(y_true * y_pred, axis = 1)
    fp = tf.reduce_sum(y_pred, axis = 1) - tp
    fn = tf.reduce_sum(y_true, axis = 1) - tp
    
    precision = tp/(tp+fp+epsilon)
    recall = tp/(tp+fn+epsilon)
    
    fb = (1+beta_squared)*precision*recall / (beta_squared*precision+recall+epsilon)
    return fb

In [15]:
# Create the model
model = Sequential() 
# 1st Convolutional Layer 
model.add(Conv2D(filters = 96, input_shape = (224, 224, 3), kernel_size = (11, 11), strides = (4, 4), padding = 'valid')) 
model.add(Activation('relu')) 
# Max-Pooling 
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid')) 
# Batch Normalisation 
model.add(BatchNormalization()) 
# 2nd Convolutional Layer 
model.add(Conv2D(filters = 256, kernel_size = (11, 11), strides = (1, 1), padding = 'valid')) 
model.add(Activation('relu')) 
# Max-Pooling 
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid')) 
# Batch Normalisation 
model.add(BatchNormalization()) 
# 3rd Convolutional Layer 
model.add(Conv2D(filters = 384, kernel_size = (3, 3), strides = (1, 1), padding = 'valid')) 
model.add(Activation('relu')) 
# Batch Normalisation 
model.add(BatchNormalization()) 
# 4th Convolutional Layer 
model.add(Conv2D(filters = 384, kernel_size = (3, 3), strides = (1, 1), padding = 'valid')) 
model.add(Activation('relu')) 
# Batch Normalisation 
model.add(BatchNormalization()) 
# 5th Convolutional Layer 
model.add(Conv2D(filters = 256, kernel_size = (3, 3), strides = (1, 1), padding = 'valid')) 
model.add(Activation('relu')) 
# Max-Pooling 
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid')) 
# Batch Normalisation 
model.add(BatchNormalization()) 
# Flattening 
model.add(Flatten()) 
# 1st Dense Layer 
model.add(Dense(4096, input_shape = (224*224*3, ))) 
model.add(Activation('relu')) 
# Add Dropout to prevent overfitting 
model.add(Dropout(0.4)) 
# Batch Normalisation 
model.add(BatchNormalization()) 
# 2nd Dense Layer 
model.add(Dense(4096)) 
model.add(Activation('relu')) 
# Add Dropout 
model.add(Dropout(0.4)) 
# Batch Normalisation 
model.add(BatchNormalization()) 
# Output Softmax Layer 
model.add(Dense(num_classes)) 
model.add(Activation('softmax'))

In [32]:
# Compile the model
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=[fbeta])

In [33]:
# Combine multiple columns into a single column with tuples
atm_train_data['labels'] = atm_train_data[['clear', 'partly_cloudy', 'cloudy', 'haze']].apply(tuple, axis=1)

atm_train_data.head()

,image_name,clear,partly_cloudy,cloudy,haze,labels
4619,train_4619.jpg,1,0,0,0,"(1, 0, 0, 0)"
14471,train_14471.jpg,1,0,0,0,"(1, 0, 0, 0)"
38436,train_38436.jpg,1,0,0,0,"(1, 0, 0, 0)"
38516,train_38516.jpg,1,0,0,0,"(1, 0, 0, 0)"
6646,train_6646.jpg,1,0,0,0,"(1, 0, 0, 0)"


In [34]:
# Initialize the ImageDataGenerator
image_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, 
                                                            shear_range=0.2,
                                                            zoom_range=0.2,
                                                            horizontal_flip=True,
                                                            validation_split=0.05)

# Create a data generator using the DataFrame
train_generator = image_gen.flow_from_dataframe(
    dataframe=atm_train_data,
    directory="train",
    x_col="image_name",
    y_col=['clear', 'partly_cloudy', 'cloudy', 'haze'],
    class_mode='raw',
    target_size=(224, 224),
    subset='training'
)

val_generator = image_gen.flow_from_dataframe(
    dataframe=atm_train_data,
    directory="train",
    x_col="image_name",
    y_col=['clear', 'partly_cloudy', 'cloudy', 'haze'],
    class_mode='raw',
    target_size=(224, 224),
    subset='validation'
)

Found 36533 validated image filenames.
Found 1922 validated image filenames.


In [35]:
model.fit(train_generator, epochs=10, batch_size=16, validation_data=val_generator)

Epoch 1/10


ValueError: in user code:

    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py", line 1081, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py", line 1139, in compute_loss
        return self.compiled_loss(
    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/compile_utils.py", line 240, in __call__
        self.build(y_pred)
    File "/opt/conda/lib/python3.10

In [26]:
# Initialize the ImageDataGenerator
test_image_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Create a data generator using the DataFrame
test_generator = image_gen.flow_from_dataframe(
    dataframe=atm_test_data,
    directory="test",
    x_col="image_name",
    y_col=['clear', 'partly_cloudy', 'cloudy', 'haze'],
    class_mode='raw',
    target_size=(224, 224)
)

pred = model.predict(test_generator)

Found 2024 validated image filenames.
64/64 [==============================] - 23s 351ms/step


In [27]:
pred

array([[9.9382448e-01, 1.0507484e-03, 4.3250716e-04, 4.6922965e-03],
       [3.9036953e-01, 1.3898051e-02, 7.6022990e-02, 5.1970941e-01],
       [6.0611272e-01, 1.5799110e-03, 8.7662742e-02, 3.0464461e-01],
       ...,
       [9.9980420e-01, 1.7714845e-04, 1.8633100e-05, 1.3757240e-07],
       [5.3394634e-01, 3.4675872e-01, 6.9555521e-02, 4.9739346e-02],
       [1.2689410e-01, 4.1182779e-02, 1.5202727e-02, 8.1672031e-01]],
      dtype=float32)

In [28]:
pred_frame = pd.DataFrame(pred, columns=['clear', 'partly_cloudy', 'cloudy', 'haze'])

# Create a DataFrame with 'image_name' and one-hot encoded columns
pred_frame['image_name'] = test_generator.filenames
pred_frame = pred_frame[['image_name', 'clear', 'partly_cloudy', 'cloudy', 'haze']]

pred_frame

,image_name,clear,partly_cloudy,cloudy,haze
0,train_34602.jpg,0.993824,0.001051,0.000433,4.692296e-03
1,train_11243.jpg,0.390370,0.013898,0.076023,5.197094e-01
2,train_14499.jpg,0.606113,0.001580,0.087663,3.046446e-01
3,train_18918.jpg,0.904271,0.010242,0.005250,8.023762e-02
4,train_30631.jpg,0.926778,0.040232,0.004053,2.893662e-02
...,...,...,...,...,...
2019,train_35963.jpg,0.001125,0.998366,0.000078,4.311022e-04
2020,train_1213.jpg,0.998610,0.000403,0.000670,3.178163e-04
2021,train_23368.jpg,0.999804,0.000177,0.000019,1.375724e-07
2022,train_499.jpg,0.533946,0.346759,0.069556,4.973935e-02


In [29]:
# Find the index of the maximum probability for each image
max_labels = np.argmax(pred, axis=1)

# Create a binary array with 1 at the index of the maximum probability
pred_binary = np.zeros_like(pred).astype(int)
pred_binary[np.arange(len(pred)), max_labels] = 1

# Create a DataFrame with 'image_name' and one-hot encoded columns
result_df = pd.DataFrame(pred_binary, columns=['clear', 'partly_cloudy', 'cloudy', 'haze'])
result_df['image_name'] = test_generator.filenames
result_df = result_df[['image_name', 'clear', 'partly_cloudy', 'cloudy', 'haze']]

result_df

,image_name,clear,partly_cloudy,cloudy,haze
0,train_34602.jpg,1,0,0,0
1,train_11243.jpg,0,0,0,1
2,train_14499.jpg,1,0,0,0
3,train_18918.jpg,1,0,0,0
4,train_30631.jpg,1,0,0,0
...,...,...,...,...,...
2019,train_35963.jpg,0,1,0,0
2020,train_1213.jpg,1,0,0,0
2021,train_23368.jpg,1,0,0,0
2022,train_499.jpg,1,0,0,0


In [31]:
# Extract the true labels from the true DataFrame
y_true = atm_test_data[['clear', 'partly_cloudy', 'cloudy', 'haze']].values

# Extract the predicted labels from the predicted DataFrame
y_pred = result_df[['clear', 'partly_cloudy', 'cloudy', 'haze']].values

# Calculate accuracy
accuracy = fbeta(y_true, y_pred)

np.sum(accuracy)/len(accuracy)

0.5192064126960845